In [ ]:
import pandas as pd

""""
    This notebook reads residential electricity and heating consumption from JRC-IDEES 2015 dataset (
    online: https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/JRC-IDEES/JRC-IDEES-2015_v1/) and alters
    electricity and heating consumption by subtracting electrified heating from heating consumption and
    adding it to electricity consumption.
"""


In [2]:
iso2name={'AT':'Austria',
          'BE':'Belgium',
          'BG':'Bulgaria',
          'HR':'Croatia',
          'CY':'Cyprus',
          'CZ':'Czechia',
          'DK':'Denmark',
          'EE':'Estonia',
          'EU28':'European Union',
          'FI':'Finland',
          'FR':'France',
          'DE':'Germany',
          'EL':'Greece',
          'HU':'Hungary',
          'IE':'Ireland',
          'IT':'Italy',
          'LV':'Latvia',
          'LT':'Lithuania',
          'LU':'Luxembourg',
         # 'NO':'Norway',
          'MT':'Malta',
          'NL':'Netherlands',
          'PL':'Poland',
          'PT':'Portugal',
          'RO':'Romania',
          'SK':'Slovakia',   
          'SI':'Slovenia',
          'ES':'Spain',
          'SE':'Sweden',
          'UK':'United Kingdom',
          #'ME':'Montenegro',
          #'AL':'Albania',
          #'BA':'Bosnia and Herz.',
          #'MK':'North Macedonia',
          #'RS':'Serbia',
         }

countries= [i for i in iso2name.keys()]
df_altered_consumption= pd.DataFrame(index= [iso2name[i] for i in countries], 
                                     columns=['electricity_consumption', 'heating_consumption'])

In [ ]:
for country in countries : 
    fec = pd.read_excel('JRC/JRC-IDEES-2015_All_xlsx_{}/JRC-IDEES-2015_Residential_{}.xlsx'.format(country, country),
                          sheet_name='RES_hh_fec',
                          index_col=0,
                          #usecols='Q',
                          #skiprows=skiprows,
                          header = 0,
                          na_values="N.A")
    
    summary = pd.read_excel('JRC/JRC-IDEES-2015_All_xlsx_{}/JRC-IDEES-2015_Residential_{}.xlsx'.format(country, country),
                          sheet_name='RES_summary',
                          index_col=0,
                          #usecols='Q',
                          #skiprows=skiprows,
                          header = 0,
                          na_values="N.A")
    
    num = pd.read_excel('JRC/JRC-IDEES-2015_All_xlsx_{}/JRC-IDEES-2015_Residential_{}.xlsx'.format(country, country),
                          sheet_name='RES_hh_num',
                          index_col=0,
                          #usecols='Q',
                          #skiprows=skiprows,
                          header = 0,
                          na_values="N.A")   
    
    fech = pd.read_excel('JRC/JRC-IDEES-2015_All_xlsx_{}/JRC-IDEES-2015_Residential_{}.xlsx'.format(country, country),
                          sheet_name='RES_hh_fech',
                          index_col=0,
                          #usecols='Q',
                          #skiprows=skiprows,
                          header = 0,
                          na_values="N.A")
    
    ## calculate total electrified heating consumption
    to_delete= [j for j in fec.index if any(i in str(j) for i in ['electric', 'Electricity','Solar',  
                                                                  'Air conditioning',
                                                              'cooling','Water heating','Cooking',
                                                              'Space heating','Final energy' ])]
    
    ## calculate heating consumption without electrified heating
    fec.loc[to_delete, 2015] =0
    energy_per_household= 1000000/0.086*fec.sum()[2015]/num.loc['Stock of households',2015]
    heat_energy_per_individual = energy_per_household/summary.loc['Households size (inhabitants/household)', 2015]
    
    ## calculate electrified heating consumption
    electrified_heating_energy_per_individual = (fech.loc['Final energy consumption (kWh / household)',2015]/
       summary.loc['Households size (inhabitants/household)', 2015]) - heat_energy_per_individual
    
     ## modify electricity consumption by ADDING electrified heating
    elec_consumption_per_individual = summary.loc['Specific electricity uses (appliances and lighting)', 2015]+(
                                     electrified_heating_energy_per_individual)
    
     ## modify heating consumption by SUBTRACTING electrified heating    
    heat_consumption_per_individual = summary.loc['Thermal uses', 2015]-(
                                     electrified_heating_energy_per_individual)
    
    df_altered_consumption.loc[iso2name[country], 'electricity_consumption'] = elec_consumption_per_individual[-2]
    df_altered_consumption.loc[iso2name[country], 'heating_consumption'] =  heat_consumption_per_individual[-2]
    

In [5]:
df_altered_consumption

,electricity_consumption,heating_consumption
Austria,2165.742145,5939.450237
Belgium,1697.788335,6730.314714
Bulgaria,1493.195726,2052.335315
Croatia,1496.153605,5158.145053
Cyprus,2682.892155,1803.593879
Czechia,1379.754925,6012.052641
Denmark,1822.062315,6918.504246
Estonia,1313.956928,6275.776041
European Union,1608.144313,4693.678292
Finland,3825.143513,6831.750081
